# Задание 1 (2 балла)

Напишите класс `MyDict`, который будет полностью повторять поведение обычного словаря, за исключением того, что при итерации мы должны получать и ключи, и значения.

**Модули использовать нельзя**

In [1]:
class MyDict(dict):    
    def __iter__(self):
        for key, val in self.items():
            yield key, val

In [2]:
dct = MyDict({"a": 1, "b": 2, "c": 3, "d": 25})
for key, value in dct:
    print(key, value)   

a 1
b 2
c 3
d 25


In [3]:
for key, value in dct.items():
    print(key, value)

a 1
b 2
c 3
d 25


In [4]:
for key in dct.keys():
    print(key)

a
b
c
d


In [5]:
dct["c"] + dct["d"]

28

# Задание 2 (2 балла)

Напишите функцию `iter_append`, которая "добавляет" новый элемент в конец итератора, возвращая итератор, который включает изначальные элементы и новый элемент. Итерироваться по итератору внутри функции нельзя, то есть вот такая штука не принимается
```python
def iter_append(iterator, item):
    lst = list(iterator) + [item]
    return iter(lst)
```

**Модули использовать нельзя**

In [6]:
def iter_append(iterator, item):
    yield from iterator
    yield item    

my_iterator = iter([1, 2, 3])
new_iterator = iter_append(my_iterator, 4)
list(new_iterator)

[1, 2, 3, 4]

In [7]:
for element in new_iterator:
    print(element)

# Задание 3 (5 баллов)

Представим, что мы установили себе некотурую библиотеку, которая содержит в себе два класса `MyString` и `MySet`, которые являются наследниками `str` и `set`, но также несут и дополнительные методы.

Проблема заключается в том, что библиотеку писали не очень аккуратные люди, поэтому получилось так, что некоторые методы возвращают не тот тип данных, который мы ожидаем. Например, `MyString().reverse()` возвращает объект класса `str`, хотя логичнее было бы ожидать объект класса `MyString`.

Найдите и реализуйте удобный способ сделать так, чтобы подобные методы возвращали экземпляр текущего класса, а не родительского. При этом **код методов изменять нельзя**

**+3 дополнительных балла** за реализацию того, чтобы **унаследованные от `str` и `set` методы** также возвращали объект интересующего нас класса (то есть `MyString.replace(..., ...)` должен возвращать `MyString`). **Переопределять методы нельзя**

**Модули использовать нельзя**

In [8]:
no_change_this_dunders = ['__class__', '__dict__', '__new__', '__str__',
                          '__repr__', '__init__', '__name__', '__module__']


def get_obj_type(method):        
    def inner_func(self, other=None):
        if other is None:
            if isinstance(method(self), (str, set)):
                result = type(self)(method(self))
            else:
                result = method(self)
        else:
            if isinstance(method(self, other), (str, set)):
                result = type(self)(method(self, other))  
            else:
                result = method(self, other)
        return result
    return inner_func

def change_obj_type(cls):
    for name_attr in dir(cls):  # find all methods of class
        if name_attr not in no_change_this_dunders:
            attr = getattr(cls, name_attr)
            setattr(cls, name_attr, get_obj_type(attr))  # change type of return value
    return cls         

In [9]:
@change_obj_type
class MyString(str):
    def reverse(self):
        return self[::-1]
    
    def make_uppercase(self):
        return "".join([chr(ord(char) - 32) if 97 <= ord(char) <= 122 else char for char in self])
    
    def make_lowercase(self):
        return "".join([chr(ord(char) + 32) if 65 <= ord(char) <= 90 else char for char in self])
    
    def capitalize_words(self):
        return " ".join([word.capitalize() for word in self.split()])


@change_obj_type
class MySet(set):
    def is_empty(self):
        return len(self) == 0
    
    def has_duplicates(self):
        return len(self) != len(set(self))
    
    def union_with(self, other):
        return self.union(other)
    
    def intersection_with(self, other):
        return self.intersection(other)
    
    def difference_with(self, other):
        return self.difference(other)

In [10]:
string_example = MyString("Aa Bb Cc")
string_example2 = MyString("Dd")
set_example_1 = MySet({1, 2, 3, 4})
set_example_2 = MySet({3, 4, 5, 6, 6})
print(type(string_example.reverse()))
print(type(string_example.make_uppercase()))
print(type(string_example.make_lowercase()))
print(type(string_example.capitalize_words()))
print(type(string_example.lower())) # method from string class
print(type(string_example.find('Bb')))
print(type(string_example.__add__(string_example2)))
print()
print(type(set_example_1.is_empty()))
print(type(set_example_2.has_duplicates()))
print(type(set_example_1.union_with(set_example_2)))
print(type(set_example_1.difference_with(set_example_2)))
print(type(set_example_1.union(set_example_2))) # method from set class
print(type(set_example_1.add(10)))  # doesn't break with return format NoneType; method(self, other)
print(set_example_1)
print(type(set_example_1.clear()))  # doesn't break with return format NoneType; method(self)

<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>
<class '__main__.MyString'>
<class 'int'>
<class '__main__.MyString'>

<class 'bool'>
<class 'bool'>
<class '__main__.MySet'>
<class '__main__.MySet'>
<class '__main__.MySet'>
<class 'NoneType'>
MySet({1, 2, 3, 4, 10})
<class 'NoneType'>


# Задание 4 (5 баллов)

Напишите декоратор `switch_privacy`:
1. Делает все публичные **методы** класса приватными
2. Делает все приватные методы класса публичными
3. Dunder методы и защищённые методы остаются без изменений
4. Должен работать тестовый код ниже, в теле класса писать код нельзя

**Модули использовать нельзя**

1. Вытащить все атрибуты
2. Заменить на новые
3. Удалить прошлые 

In [11]:
def swich_property(cls):
    for name_attr in dir(cls):
        if name_attr[0] != '_':
            renamed_method = f'_{cls.__name__}__' + name_attr
        elif name_attr.startswith(f'_{cls.__name__}__'):
            renamed_method = name_attr.split('__')[1]
        else:
            continue
        attr = getattr(cls, name_attr)
        setattr(cls, renamed_method, attr)
        delattr(cls, name_attr)
    
    return cls

In [12]:
@swich_property 
class ExampleClass:
    # Но не здесь
    
    def public_method(self):
        return 1
    
    def _protected_method(self):
        return 2
    
    def __private_method(self):
        return 3
    
    def __dunder_method__(self):
        pass

In [13]:
test_object = ExampleClass()
test_object._ExampleClass__public_method()   # Публичный метод стал приватным

1

In [14]:
test_object.private_method()   # Приватный метод стал публичным

3

In [15]:
test_object._protected_method()   # Защищённый метод остался защищённым

2

In [16]:
test_object.__dunder_method__()   # Дандер метод не изменился

In [17]:
hasattr(test_object, "public_method"), hasattr(test_object, "private")   # Изначальные варианты изменённых методов не сохраняются


(False, False)

# Задание 5 (7 баллов)

Напишите [контекстный менеджер](https://docs.python.org/3/library/stdtypes.html#context-manager-types) `OpenFasta`

Контекстные менеджеры это специальные объекты, которые могут работать с конструкцией `with ... as ...:`. В них нет ничего сложного, для их реализации как обычно нужно только определить только пару dunder методов. Изучите этот вопрос самостоятельно

1. Объект должен работать как обычные файлы в питоне (наследоваться не надо, здесь лучше будет использовать **композицию**), но:
    + При итерации по объекту мы должны будем получать не строку из файла, а специальный объект `FastaRecord`. Он будет хранить в себе информацию о последовательности. Важно, **не строки, а именно последовательности**, в fasta файлах последовательность часто разбивают на много строк
    + Нужно написать методы `read_record` и `read_records`, которые по смыслу соответствуют `readline()` и `readlines()` в обычных файлах, но они должны выдавать не строки, а объект(ы) `FastaRecord`
2. Конструктор должен принимать один аргумент - **путь к файлу**
3. Класс должен эффективно распоряжаться памятью, с расчётом на работу с очень большими файлами
    
Объект `FastaRecord`. Это должен быть **датакласс** (см. про примеры декораторов в соответствующей лекции) с тремя полями:
+ `seq` - последовательность
+ `id_` - ID последовательности (это то, что в фаста файле в строке, которая начинается с `>` до первого пробела. Например, >**GTD326487.1** Species anonymous 24 chromosome) 
+ `description` - то, что осталось после ID (Например, >GTD326487.1 **Species anonymous 24 chromosome**)


Напишите демонстрацию работы кода с использованием всех написанных методов, обязательно добавьте файл с тестовыми данными в репозиторий (не обязательно большой)

**Можно использовать модули из стандартной библиотеки**

In [2]:
from dataclasses import dataclass
import os
import re

In [3]:
@dataclass
class FastaRecord:
    seq: str
    id_: str
    description: str


class OpenFasta:
    def __init__(self, file_name):
        self._fasta_record = FastaRecord(seq='', id_='', description='')
        self.file_obj = open(file_name)
        self.__seq_l = []

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        self.file_obj.close()

    def __iter__(self):
        yield self.file_obj

    def read_record(self):
        counter_name_seq = 0
        self.__seq_l.clear()
        while counter_name_seq <= 2:
            line = next(self.file_obj)
            line = line.strip('\n')
            if line.startswith('#'):
                continue
            
            elif line.startswith('>'):
                if self._fasta_record.id_ == '':  # need for first line: >gene spesies
                    self._fasta_record.id_, self._fasta_record.description = re.findall(r'(.+?) (.+)', line)[0]
                else:
                    self._fasta_record.seq = ''.join(self.__seq_l)
                    # make object with current values
                    current_fasta_record = self._fasta_record
                    # clear current info
                    self._fasta_record = FastaRecord(seq='', id_='', description='')
                    # remember for next iteration
                    self._fasta_record.id_, self._fasta_record.description = re.findall(r'(.+?) (.+)', line)[0]
                    return current_fasta_record
                
            else:
                self.__seq_l.append(line)

    def read_records(self):
        result_all_fasta = []
        while True:
            try:
                result_all_fasta.append(self.read_record())
            except StopIteration:
                # write last sequences
                self._fasta_record.seq = ''.join(self.__seq_l)
                result_all_fasta.append(self._fasta_record)
                return result_all_fasta

In [4]:
with OpenFasta(os.path.join('data/', 'test_fasta.fasta')) as fasta:
    first_record = fasta.read_record()
    remaining_records = fasta.read_records()

In [5]:
print(first_record)

FastaRecord(seq='ATGGGCGATCTTGCTATGTCCGTAGCAGACATCAGGATGGAGAATGAGCCTGATGATTTAGCTAGTGATAATGTTGCTGAGATTGATGTGAGTGATGAAGAGATTGATGCTGACGACCTTGAGAGACGGATGTGGAAAGATCGTGTCAGGCTTAAAAGAATCAAAGAGCGACAAAAAGCTGGCTCTCAAGGA', id_='>AT0001', description='Arabidosis thaliana')


In [6]:
print(remaining_records)

[FastaRecord(seq='ATGTACGGAGACGCTACAAACTGGAATGAAGATGAGTATAGAGAATCAATTTTGAAGGAGCGAGAGATAGAGACACGCACCGTCTTCAGAACCGCCTGGGCTCCTCCGGCGAGAATCTCTAATCCAGACGCATTTGTTG', id_='>AT0002', description='Arabidosis thaliana'), FastaRecord(seq='ATGTACGGAGACGCTACAAACTGGAATGAAGATGAGTATAGAGAATCAATTTTGAAGGAGCGAGAGATAGAGAC', id_='>AT0003', description='Arabidosis thaliana')]


In [7]:
with OpenFasta(os.path.join('data/', 'test_fasta.fasta')) as fasta:
    remaining_records = fasta.read_records()
print(remaining_records)

[FastaRecord(seq='ATGGGCGATCTTGCTATGTCCGTAGCAGACATCAGGATGGAGAATGAGCCTGATGATTTAGCTAGTGATAATGTTGCTGAGATTGATGTGAGTGATGAAGAGATTGATGCTGACGACCTTGAGAGACGGATGTGGAAAGATCGTGTCAGGCTTAAAAGAATCAAAGAGCGACAAAAAGCTGGCTCTCAAGGA', id_='>AT0001', description='Arabidosis thaliana'), FastaRecord(seq='ATGTACGGAGACGCTACAAACTGGAATGAAGATGAGTATAGAGAATCAATTTTGAAGGAGCGAGAGATAGAGACACGCACCGTCTTCAGAACCGCCTGGGCTCCTCCGGCGAGAATCTCTAATCCAGACGCATTTGTTG', id_='>AT0002', description='Arabidosis thaliana'), FastaRecord(seq='ATGTACGGAGACGCTACAAACTGGAATGAAGATGAGTATAGAGAATCAATTTTGAAGGAGCGAGAGATAGAGAC', id_='>AT0003', description='Arabidosis thaliana')]


# Задание 6 (7 баллов)

1. Напишите код, который позволит получать все возможные (неуникальные) генотипы при скрещивании двух организмов. Это может быть функция или класс, что вам кажется более удобным.

Например, все возможные исходы скрещивания "Aabb" и "Aabb" (неуникальные) это

```
AAbb
AAbb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
aabb
aabb
```

2. Напишите функцию, которая вычисляет вероятность появления определённого генотипа (его ожидаемую долю в потомстве).
Например,

```python
get_offspting_genotype_probability(parent1="Aabb", parent2="Aabb", target_genotype="Аabb")   # 0.5

```

3. Напишите код, который выводит все уникальные генотипы при скрещивании `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` и `'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН'`, которые содержат в себе следующую комбинацию аллелей `'АаБбВвГгДдЕеЖжЗзИиЙйКкЛл'`
4. Напишите код, который расчитывает вероятность появления генотипа `'АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'` при скрещивании `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн` и `АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн`

Важные замечания:
1. Порядок следования аллелей в случае гетерозигот всегда должен быть следующим: сначала большая буква, затем маленькая (вариант `AaBb` допустим, но `aAbB` быть не должно)
2. Подзадачи 3 и 4 могут потребовать много вычислительного времени (до 15+ минут в зависимости от железа), поэтому убедитесь, что вы хорошо протестировали написанный вами код на малых данных перед выполнением этих задач. Если ваш код работает **дольше 20 мин**, то скорее всего ваше решение не оптимально, попытайтесь что-нибудь оптимизировать. Если оптимальное решение совсем не получается, то попробуйте из входных данных во всех заданиях убрать последний ген (это должно уменьшить время выполнения примерно в 4 раза), но **за такое решение будет снято 2 балла**
3. Несмотря на то, что подзадания 2, 3 и 4 возможно решить математически, не прибегая к непосредственному получению всех возможных генотипов, от вас требуется именно brute-force вариант алгоритма

**Можно использовать модули из стандартной библиотеки питона**, но **за выполнение задания без использования модулей придусмотрено +3 дополнительных балла**

In [24]:
from datetime import datetime

In [25]:
class CrossingGenotypes:
    def __init__(self, genotype1, genotype2):
        self._split_genotype1 = self._split_by_chromosome_types(genotype1)
        self._split_genotype2 = self._split_by_chromosome_types(genotype2)
    
    @staticmethod
    def _split_by_chromosome_types(genotype):
        splitted_chromosomes = []
        possible_combinations = [[]]

        lower_genotype = genotype.lower()
        ploid = lower_genotype.count(lower_genotype[0])
        
        for idx in range(0, len(genotype) - 1, ploid):
            splitted_chromosomes.append([genotype[idx], genotype[idx+1]])
        
        for chroms in splitted_chromosomes:
            possible_combinations = [x+[y] for x in possible_combinations for y in chroms]  # based on itertools.product()
        return possible_combinations
    
    def print_offspting(self):
        for haplotype1 in self._split_genotype1:
            for haplotype2 in self._split_genotype2:
                new_possible_genotype = ''.join(sorted(sorted(haplotype1 + haplotype2), key=str.lower))
                print(new_possible_genotype)
    
    def get_offspting_genotype_probability(self, offspring):
        count_matches = 0
        count_all_genotypes = 0
        
        for haplotype1 in self._split_genotype1:
            for haplotype2 in self._split_genotype2:
                count_all_genotypes += 1
                new_possible_genotype = ''.join(sorted(sorted(haplotype1 + haplotype2), key=str.lower)) 
                if new_possible_genotype == offspring:
                    count_matches += 1
        return count_matches / count_all_genotypes
    
    def get_unique_genotypes_with_substring(self, substring):
        remember_genotypes = []
        
        for haplotype1 in self._split_genotype1:
            for haplotype2 in self._split_genotype2:
                new_possible_genotype = ''.join(sorted(sorted(haplotype1 + haplotype2), key=str.lower))
                if substring in new_possible_genotype and new_possible_genotype not in remember_genotypes:
                    remember_genotypes.append(new_possible_genotype)
                    print(new_possible_genotype)
      

In [26]:
# 1, 2
genotype = CrossingGenotypes('Aabb', 'Aabb')
genotype.print_offspting()
print(genotype.get_offspting_genotype_probability('Aabb'))

AAbb
AAbb
Aabb
Aabb
AAbb
AAbb
Aabb
Aabb
Aabb
Aabb
aabb
aabb
Aabb
Aabb
aabb
aabb
0.5


In [27]:
# 3
start = datetime.now()
lots_of_chromosomes = CrossingGenotypes('АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн', 'АаббВвГгДДЕеЖжЗзИиЙйКкЛлМмНН')
lots_of_chromosomes.get_unique_genotypes_with_substring('АаБбВвГгДдЕеЖжЗзИиЙйКкЛл')
stop = datetime.now()

АаБбВвГгДдЕеЖжЗзИиЙйКкЛлММНН
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНН
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлММНн
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлммНН
АаБбВвГгДдЕеЖжЗзИиЙйКкЛлммНн


In [28]:
print(stop - start)  # in PyCharm works 10-11 minutes

0:18:06.789252


In [31]:
# 4
start = datetime.now()
lots_of_chromosomes = CrossingGenotypes('АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн', 'АаБбВвГгДдЕеЖжЗзИиЙйКкЛлМмНн')
print(lots_of_chromosomes.get_offspting_genotype_probability('АаБбввГгДдЕеЖжЗзИиЙйккЛлМмНн'))
stop = datetime.now()

6.103515625e-05


In [32]:
print(stop - start)  # in PyCharm works 10-11 minutes

0:19:03.065887
